In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-alu84xil
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-alu84xil
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [2]:
!pip install opencv-python ftfy regex tqdm

In [3]:
import torch
import clip
from PIL import Image

# 加載模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)



100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 53.8MiB/s]


In [ ]:
# 加載並預處理圖片
image_path = "images.jpg"  # 替換成你圖片的路徑
image = Image.open(image_path)
image_input = preprocess(image).unsqueeze(0).to(device)

# 定義可能的文本描述
text_descriptions = [
    "punch/slap",
    "kicking",
    "pushing",
    "pat on back",
    "point finger",
    "hugging",
    "walking apart",
    "hit with object",
    "standing"
]
text_inputs = torch.cat([clip.tokenize(desc) for desc in text_descriptions]).to(device)

# 計算圖片和文本特徵
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# 計算圖片和每個文本描述之間的相似性
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (image_features @ text_features.T).squeeze(0)

# 找到最相似的描述
best_match = text_descriptions[similarity.argmax().item()]
print(best_match)

waving left hand


In [7]:
import torch
import clip
from PIL import Image
import cv2
import numpy as np

# 加载CLIP模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 定义可能的文本描述
text_descriptions = [
    "drink water",
    "dancing",
    "kicking something",
    "shaking hands",
    "walking apart",
    "pat on back",
    "punch/slap",
    "kicking",
    "pushing",
    "point finger",
    "hugging",
    "hit with object",
    "standing",
    "looking each other"
]
text_inputs = torch.cat([clip.tokenize(desc) for desc in text_descriptions]).to(device)

# 打开camera
cap = cv2.VideoCapture("/home/user/UCF-101/HandstandWalking/v_HandstandWalking_g08_c02.avi")

frame_count = 0
frame_interval = 1  # 每隔帧处理一帧
similarity_threshold = 0.23  # 设置相似度阈值
accumulated_similarity = torch.zeros(len(text_descriptions), device=device)
window_size = 1  # 考虑的帧数

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    if frame_count % frame_interval == 0:
        # 将帧转换为PIL图像
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        # 预处理图片并计算特征
        image_input = preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image_input)
        # 计算图片和每个文本描述之间的相似性
        image_features /= image_features.norm(dim=-1, keepdim=True)
        with torch.no_grad():
            text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = (image_features @ text_features.T).squeeze(0)
        # 累积相似度
        accumulated_similarity += similarity
        # 每 window_size 帧后进行判断
        if frame_count % (frame_interval * window_size) == 0:
            # 计算平均相似度
            avg_similarity = accumulated_similarity / window_size
            #print(avg_similarity)
            # 找到最相似的前五个描述
            top_k = 5
            top_k_indices = avg_similarity.topk(top_k).indices
            best_matches = [text_descriptions[i] for i in top_k_indices]

            # 确认相似度是否高于阈值
            if avg_similarity[top_k_indices[0]] < similarity_threshold:
                display_text = "Action not recognized"
            else:
                display_text = ', '.join(best_matches)

            print(f"Frame {frame_count}: {display_text}")

            cv2.putText(frame, best_matches[0], (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, best_matches[1], (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, best_matches[2], (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 1, cv2.LINE_AA)
            cv2.imshow("Action Recognition", frame)

            # 重置累积相似度
            accumulated_similarity = torch.zeros(len(text_descriptions), device=device)

        if cv2.waitKey(1) == ord('q'): # 按 'q' 键退出
            break

cap.release()
cv2.destroyAllWindows()

Frame 2: shaking hands
Frame 4: shaking hands
Frame 6: standing
Frame 8: shaking hands
Frame 10: shaking hands
Frame 12: shaking hands
Frame 14: shaking hands
Frame 16: shaking hands
Frame 18: shaking hands
Frame 20: shaking hands
Frame 22: shaking hands
Frame 24: hit with object
Frame 26: standing
Frame 28: shaking hands
Frame 30: shaking hands
Frame 32: shaking hands
Frame 34: shaking hands
